# Run easyocr

In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install easyocr fire lmdb opencv-python natsort nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 25.7 MB/s eta 0:00:00


In [3]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118


In [4]:
%cd /content/drive/MyDrive/Ai_Lab/Hand_Writing/models_github/deep-text-recognition-benchmark-master

/content/drive/MyDrive/Ai_Lab/Hand_Writing/models_github/deep-text-recognition-benchmark-master


In [ ]:
!python3 create_lmdb_dataset.py \
  --inputPath output \
  --gtFile output/labels.txt \
  --outputPath lmdb_output

Created dataset with 16 samples


# Show LMDB

In [ ]:
import lmdb
import cv2
import numpy as np
from google.colab import drive
import os

In [ ]:

# مسیر دیتابیس LMDB
lmdb_path = "/content/drive/MyDrive/Ai_Lab/Hand_Writing/models_github/deep-text-recognition-benchmark-master/lmdb_output"

# باز کردن دیتابیس LMDB
env = lmdb.open(lmdb_path, readonly=True, lock=False)

# خواندن تعداد نمونه‌ها
with env.begin() as txn:
    num_samples = int(txn.get('num-samples'.encode()).decode())
    print(f"Number of samples: {num_samples}")

# خواندن تصاویر و برچسب‌ها
images = []
labels = []

with env.begin() as txn:
    for i in range(1, num_samples + 1):
        # کلیدهای تصویر و برچسب
        image_key = f'image-{i:09d}'.encode()
        label_key = f'label-{i:09d}'.encode()

        # خواندن داده باینری تصویر
        image_bin = txn.get(image_key)
        if image_bin is None:
            print(f"Image {image_key.decode()} not found")
            continue

        # خواندن برچسب
        label = txn.get(label_key).decode()
        if label is None:
            print(f"Label {label_key.decode()} not found")
            continue

        # تبدیل داده باینری به تصویر
        image_buf = np.frombuffer(image_bin, dtype=np.uint8)
        image = cv2.imdecode(image_buf, cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"Failed to decode image {image_key.decode()}")
            continue

        images.append(image)
        labels.append(label)
        print(f"Image {i}: {label}")

# نمایش یا ذخیره تصاویر (اختیاری)
output_dir = "/content/drive/MyDrive/Ai_Lab/Hand_Writing/models_github/deep-text-recognition-benchmark-master/show_er/extracted_images"
os.makedirs(output_dir, exist_ok=True)

for i, (image, label) in enumerate(zip(images, labels)):
    output_path = os.path.join(output_dir, f"extracted_image_{i+1}.png")
    cv2.imwrite(output_path, image)
    print(f"Saved {output_path} with label: {label}")

# ذخیره برچسب‌ها در یک فایل (اختیاری)
with open("/content/drive/MyDrive/Ai_Lab/Hand_Writing/models_github/deep-text-recognition-benchmark-master/show_er/extracted_labels.txt", "w") as f:
    for i, label in enumerate(labels):
        f.write(f"image_{i+1}.png {label}\n")

print("Extraction completed!")

Number of samples: 16
Image 1: Testing
Image 2: Easy
Image 3: Valid
Image 4: Manchester
Image 5: England
Image 6: Must
Image 7: Nice
Image 8: Is
Image 9: Simple
Image 10: New York
Image 11: OCR
Image 12: USA
Image 13: To
Image 14: Norway
Image 15: Does
Image 16: Essay
Saved /content/drive/MyDrive/Ai_Lab/Hand_Writing/models_github/deep-text-recognition-benchmark-master/show_er/extracted_images/extracted_image_1.png with label: Testing
Saved /content/drive/MyDrive/Ai_Lab/Hand_Writing/models_github/deep-text-recognition-benchmark-master/show_er/extracted_images/extracted_image_2.png with label: Easy
Saved /content/drive/MyDrive/Ai_Lab/Hand_Writing/models_github/deep-text-recognition-benchmark-master/show_er/extracted_images/extracted_image_3.png with label: Valid
Saved /content/drive/MyDrive/Ai_Lab/Hand_Writing/models_github/deep-text-recognition-benchmark-master/show_er/extracted_images/extracted_image_4.png with label: Manchester
Saved /content/drive/MyDrive/Ai_Lab/Hand_Writing/models_g

# Finr Tune

In [6]:

p = 'dataset.py'
src = open(p, 'r', encoding='utf-8').read()

old = 'from torch._utils import _accumulate'
new = (
    'try:\n'
    '    from torch._utils import _accumulate  # PyTorch < 2.2\n'
    'except Exception:\n'
    '    # PyTorch >= 2.2: define a small fallback\n'
    '    def _accumulate(iterable):\n'
    '        total = 0\n'
    '        for x in iterable:\n'
    '            total += x\n'
    '            yield total\n'
)

if old in src:
    src = src.replace(old, new)
    open(p, 'w', encoding='utf-8').write(src)
    print('Patched dataset.py ✅')
else:
    print('dataset.py already patched or different import line.')


Patched dataset.py ✅


In [10]:
!python3 train.py \
  --train_data  lmdb_output \
  --valid_data  lmdb_output \
  --select_data   "/" \
  --batch_ratio     1.0 \
  --Transformation    TPS \
  --FeatureExtraction    ResNet \
  --SequenceModeling    BiLSTM \
  --Prediction    Attn \
  --batch_size    2 \
  --data_filtering_off  \
  --workers    0 \
  --batch_max_length    80 \
  --num_iter    10 \
  --valInterval    5 \
  --saved_model   TPS-ResNet-BiLSTM-Attn.pth

--------------------------------------------------------------------------------
dataset_root: lmdb_output
opt.select_data: ['/']
opt.batch_ratio: ['1.0']
--------------------------------------------------------------------------------
dataset_root:    lmdb_output	 dataset: /
sub-directory:	/.	 num samples: 16
num total samples of /: 16 x 1.0 (total_data_usage_ratio) = 16
num samples of / per batch: 2 x 1.0 (batch_ratio) = 2
--------------------------------------------------------------------------------
Total_batch_size: 2 = 2
--------------------------------------------------------------------------------
dataset_root:    lmdb_output	 dataset: /
sub-directory:	/.	 num samples: 16
--------------------------------------------------------------------------------
model input parameters 32 100 20 1 512 256 38 80 TPS ResNet BiLSTM Attn
Skip Transformation.LocalizationNetwork.localization_fc2.weight as it is already initialized
Skip Transformation.LocalizationNetwork.localization_fc2.bias a

# Inference with Fine-tuned Model

In [13]:
!python3 demo.py \
  --Transformation   TPS \
  --FeatureExtraction   ResNet \
  --SequenceModeling   BiLSTM \
  --Prediction   Attn \
  --image_folder   er/test \
  --saved_model   saved_models/TPS-ResNet-BiLSTM-Attn-Seed1111/best_accuracy.pth


model input parameters 32 100 20 1 512 256 38 25 TPS ResNet BiLSTM Attn
loading pretrained model from saved_models/TPS-ResNet-BiLSTM-Attn-Seed1111/best_accuracy.pth
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
--------------------------------------------------------------------------------
image_path               	predicted_labels         	confidence score
--------------------------------------------------------------------------------
er/test/image0.png       	testing                  	0.9999
er/test/image1.png       	easy                     	1.0000
er/test/image2.png       	valid

# Show Output